In [70]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import floor, ceil
from pylab import rcParams
import re

%matplotlib inline

In [71]:
#Write code to convert CSV to Pandas. Pretty sure I lost it somewhere
test = pd.read_csv('./test.csv')
baseTrain = pd.read_csv('./train.csv')

In [72]:
baseTrain

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
5,1,1,2010-03-12,21043.39,False
6,1,1,2010-03-19,22136.64,False
7,1,1,2010-03-26,26229.21,False
8,1,1,2010-04-02,57258.43,False
9,1,1,2010-04-09,42960.91,False


In [73]:
baseTrain_y = baseTrain['Weekly_Sales']
baseTrain_x = baseTrain.drop('Weekly_Sales', axis = 1)

In [74]:
baseTrain_x['Date'] = baseTrain_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
baseTrain_x['Date'] = baseTrain_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)

In [75]:
baseTrain_x

,Store,Dept,Date,IsHoliday
0,1,1,12653.568,False
1,1,1,12659.616,True
2,1,1,12665.664,False
3,1,1,12671.712,False
4,1,1,12677.760,False
5,1,1,12683.808,False
6,1,1,12689.820,False
7,1,1,12695.868,False
8,1,1,12701.916,False
9,1,1,12707.964,False


In [76]:
baseTrain_y

0         24924.50
1         46039.49
2         41595.55
3         19403.54
4         21827.90
5         21043.39
6         22136.64
7         26229.21
8         57258.43
9         42960.91
10        17596.96
11        16145.35
12        16555.11
13        17413.94
14        18926.74
15        14773.04
16        15580.43
17        17558.09
18        16637.62
19        16216.27
20        16328.72
21        16333.14
22        17688.76
23        17150.84
24        15360.45
25        15381.82
26        17508.41
27        15536.40
28        15740.13
29        15793.87
            ...   
421540      778.70
421541      559.14
421542      605.80
421543      619.41
421544      694.25
421545      893.60
421546      745.44
421547      795.94
421548      874.64
421549      713.50
421550      856.35
421551      622.62
421552      690.52
421553      659.65
421554      695.21
421555      845.30
421556      657.63
421557      516.46
421558      727.49
421559      500.16
421560      415.40
421561      

In [77]:
#Appending all possible features together
engine = sqlite3.connect('DB')

storesWithTrain = pd.read_sql("""
SELECT *
FROM trainDf
JOIN storesDf 
ON storesDf.Store = trainDf.Store
""", engine)

_, i = np.unique(storesWithTrain.columns, return_index = True)
storesWithTrain = storesWithTrain.iloc[:, i]
storesWithTrain.to_sql('storesWithTrain', engine, if_exists='replace', index = False)

everything = pd.read_sql("""
SELECT *
FROM storesWithTrain
JOIN featuresDf
ON storesWithTrain.Store = featuresDf.Store AND storesWithTrain.Date = featuresDf.Date
""", engine)

#Get all unique elements
_, i = np.unique(everything.columns, return_index = True)
everything = everything.iloc[:, i]

In [78]:
#Parsing the data/convert data time to milliseconds
train_x = everything.drop_duplicates()
train_x = train_x.dropna().reset_index(drop = True)#Get rid of this line if you don't want to drop NaN
#Convert date string to date time object to get milliseconds
train_x['Date'] = train_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
train_x['Date'] = train_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)

test['Date'] = test['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
test['Date'] = test['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 100000)

train_y = train_x['Weekly_Sales']
train_y = train_y.to_frame()
train_y = train_y.dropna().reset_index(drop = True)
train_x.drop('Weekly_Sales', axis = 1, inplace = True)

In [79]:
#Optional cell if you want to remove NaNs. This will mainly remove the MarkDown rows
#Also changed the store types from A and B to 1 and 2. Or could one hot encode
storeType = train_x['Type']
encoding = pd.get_dummies(storeType)
#encoding = encoding.drop('C', axis = 1)

train_x['A'] = encoding['A']
train_x['B'] = encoding['B']
train_x['C'] = encoding['C']
train_x.drop('Type', axis = 1, inplace = True)
train_x = train_x.dropna().reset_index(drop = True)

In [81]:
baseTrain_x

,Store,Dept,Date,IsHoliday
0,1,1,12653.568,False
1,1,1,12659.616,True
2,1,1,12665.664,False
3,1,1,12671.712,False
4,1,1,12677.760,False
5,1,1,12683.808,False
6,1,1,12689.820,False
7,1,1,12695.868,False
8,1,1,12701.916,False
9,1,1,12707.964,False


In [80]:
train_x.to_sql('train_x', engine, if_exists = 'replace', index = False)
train_y.to_sql('train_y', engine, if_exists = 'replace', index = False)
test.to_sql('test', engine, if_exists='replace', index = False)
baseTrain_x.to_sql('baseTrain_x', engine, if_exists='replace', index = False)
baseTrain_y.to_sql('baseTrain_y', engine, if_exists='replace', index = False)